In [ ]:
from DocumentProcessing.pdf_processing import pdf_processor
from DocumentProcessing.pdf_processing import paper_processor

In [ ]:
pdf_paper = "/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/1810.04805.pdf"

In [ ]:
full_text, title,publish_display_data, authors, abstract, references = paper_processor.extract_text_from_pdf(pdf_paper)

In [ ]:
full_text = pdf_processor.extract_text(pdf_paper)

In [ ]:
full_text[1]

In [ ]:
pdf_processor.extract_toc_from_pdf(pdf_paper)

In [ ]:
full_text_2[3]

In [ ]:
pdf_processor.extract_pdf_metadata(pdf_paper)

In [ ]:
# pdf_processor.py

import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path, max_non_text_pages=10):
    with fitz.open(pdf_path) as doc:
        text = ''
        non_text_page_count = 0

        for page in doc:
            page_text = page.get_text()
            if not page_text:
                non_text_page_count += 1
                if non_text_page_count >= max_non_text_pages:
                    raise ValueError("PDF contains too many non-text pages. Please use your own OCR tool first.")
                continue
            else:
                non_text_page_count = 0  # Reset counter if a text page is found
            text += page_text

        return text

def extract_toc_from_pdf(pdf_path, max_non_text_pages=10):
    with fitz.open(pdf_path) as doc:
        toc = doc.get_toc(simple=False)
        non_text_page_count = 0

        # Check for non-text pages in the document
        for page in doc:
            if not page.get_text():
                non_text_page_count += 1
                if non_text_page_count >= max_non_text_pages:
                    raise ValueError("PDF contains too many non-text pages. Please use your own OCR tool first.")
                continue
            else:
                non_text_page_count = 0  # Reset counter if a text page is found
                break  # Break the loop after finding the first text page

        return toc


def extract_notes_from_pdf(pdf_path):
    notes = []
    with fitz.open(pdf_path) as doc:
        for page in doc:
            annotations = page.annots()
            if annotations:
                for annot in annotations:
                    annot_text = annot.info['content']
                    if annot_text:
                        notes.append(annot_text)
    return notes


def extract_pdf_metadata(pdf_path):
    with fitz.open(pdf_path) as doc:
        metadata = doc.metadata
    return metadata

# Example usage
if __name__ == "__main__":
    pass


def extract_pdf_metadata(file_path):
    """
    Extracts metadata from a PDF file.
    Args:
    file_path (str): Path to the PDF file.
    Returns:
    dict: Metadata of the PDF.
    """
    metadata = {}
    with pdfplumber.open(file_path) as pdf:
        metadata = pdf.metadata
    return metadata

In [ ]:
from DocumentManagement.Elastic import IndexSettingsGenerator,search_engine

In [ ]:
from elasticsearch import Elasticsearch, helpers

In [ ]:
es = Elasticsearch(
    ['https://localhost:9200'],
    http_auth=('elastic', 'dR8dVIqQ5=i3pPSH00zC'),  # Replace with your credentials
    verify_certs=True,
    ca_certs='/root/http_ca.crt'  # Path to your CA certificate
)


In [ ]:
from tika import parser

parsed = parser.from_file(pdf_paper)
print(parsed["metadata"])  # To print the metadata of the document.
print(parsed["content"])   # To print the text content of the document.


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)

    # Extract the content
    content = parsed['content']
    if not content:
        raise ValueError("No content found in the PDF file.")

    # Split the content into pages
    pages = content.split('\f')  # Splitting by form feed character which typically represents page breaks
    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, 1):
        # Clean up whitespace
        page_text = ' '.join(page_content.strip().split())

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("PDF seems to contain only images. Please use OCR first.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text

# Usage
# file_path = 'path_to_your_pdf_file.pdf'
# pdf_text = extract_text_tika(file_path)


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)

    # Extract the content
    content = parsed['content']
    if not content:
        raise ValueError("No content found in the PDF file.")

    # Split the content into pages
    pages = content.split('\f')  # Splitting by form feed character which typically represents page breaks
    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, 1):
        # Clean up whitespace
        page_text = ' '.join(page_content.strip().split())

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("PDF seems to contain only images. Please use OCR first.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text

# Usage
# file_path = 'path_to_your_pdf_file.pdf'
# pdf_text = extract_text_tika(file_path)


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)
    content = parsed['content']

    # Split the content by pages - Tika's plain text output may not provide clear page delimiters
    # Adjust the splitting logic based on the actual output format
    pages = content.split('some_page_delimiter')  # Replace 'some_page_delimiter' with the actual delimiter used by Tika

    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, start=1):
        page_text = page_content.strip()

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("10 consecutive pages without text found. PDF may contain only images. Please use OCR.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text


In [ ]:
pdf_text = extract_text_tika(pdf_paper)

In [ ]:
pdf_paper

In [ ]:
pdf_text

In [ ]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO

def add_invisible_markers(pdf_path, marker):
    reader = PdfReader(pdf_path)
    writer = PdfWriter()

    for i in range(len(reader.pages)):
        page = reader.pages[i]
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=letter)
        can.setFontSize(1)  # Setting font size to 1 or 0 to make it nearly invisible
        can.drawString(0, 0, marker)  # Position at bottom-left; can adjust as needed
        can.save()

        packet.seek(0)
        new_pdf = PdfReader(packet)
        page.merge_page(new_pdf.pages[0])
        writer.add_page(page)

    with open("/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/modified_pdf.pdf", "wb") as f:
        writer.write(f)

add_invisible_markers(pdf_paper ,"__PageBreak__12345__")



In [ ]:
from tika import parser

def extract_pages_with_tika(file_path, marker):
    parsed = parser.from_file(file_path)
    content = parsed['content']
    pages = content.split(marker)  # Splitting text using the marker

    text_by_page = {i+1: page.strip() for i, page in enumerate(pages)}
    return text_by_page

text_by_page = extract_pages_with_tika("/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/modified_pdf.pdf", "__PageBreak__12345__")


In [ ]:
text_by_page

In [ ]:
full_text = pdf_processor.extract_text(pdf_paper)

In [ ]:
text_by_page[1], full_text[1]

In [2]:
from DocumentIndexing.Elastic import IndexSettingsGenerator
from DocumentIndexing.Elastic import search_engine
from elasticsearch.helpers import scan

In [3]:
from config import *

In [4]:
from DocumentIndexing.Embedding import embedding_local

/root/codebox39/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
indexsetting = IndexSettingsGenerator.GeneralIndexSettings(EMBEDDING_DINENSION)
mapping = indexsetting.get_base_mapping()

In [5]:
mapping

{'mappings': {'properties': {'document_id_elastic': {'type': 'keyword'},
   'document_id_universal': {'type': 'keyword'},
   'upload_date': {'type': 'date', 'format': 'epoch_millis'},
   'original_page_number': {'type': 'integer'},
   'text_piece': {'type': 'text'},
   'language': {'type': 'keyword'},
   'text_piece_vector': {'type': 'dense_vector', 'dims': 512},
   'metadata': {'type': 'object', 'dynamic': True}}}}

In [5]:
eleasticengine = search_engine.SearchEngine()

In [ ]:
for index_name in DOCUMENT_TYPE:
    eleasticengine.create_index(index_name,mapping)

In [ ]:
index_name = "research_paper"

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

# Connect to Elasticsearch
es = Elasticsearch(hosts=ES_HOST, http_auth=ES_HTTP_AUTH, verify_certs=True, ca_certs=ES_CA_CERTS_PATH)
# Index name
index_name = "research_paper"

# Initialize the scan
results = scan(es, index=index_name, query={"query": {"match_all": {}}})

# Iterate over the results
for doc in results:
    print(doc)


In [19]:
res = eleasticengine.search_for_terms("research_paper",'finetune',language='en')

In [24]:
res['hits']['hits'][0]

{'_index': 'research_paper',
 '_id': '2610a7e7-359f-4e79-85d8-d3e3dff51dab_16_0',
 '_score': 4.584983,
 '_source': {'document_id_universal': '2610a7e7-359f-4e79-85d8-d3e3dff51dab',
  'upload_date': 1702506147687,
  'language': 'en',
  'metadata': {'Author': '',
   'ModDate': '28-05-2019',
   'Keywords': '',
   'Title': '',
   'Subject': ''},
  'document_id_elastic': '2610a7e7-359f-4e79-85d8-d3e3dff51dab_16_0',
  'original_page_number': 16,
  'text_piece': 'Note that the purpose of the masking strategies\nis to reduce the mismatch between pretraining\nand ﬁnetuning as the MASK symbol never ap\npears during the ﬁnetuning stage We report the\nDev results for both MNLI and NER For NER\nwe report both ﬁnetuning and featurebased ap\nproaches as we expect the mismatch will be am\npliﬁed for the featurebased approach as the model\nwill not have the chance to adjust the representa\ntions\nMasking Rates\nDev Set Results\nMASK SAME RND MNLI\nNER\nFinetune Finetune Featurebased\n80 10 10\n100\n 20

In [13]:
vector = embedding_local.embeddings_multilingual("ﬁnetuning")

In [14]:
res2 = eleasticengine.vector_search("research_paper",vector)

In [15]:
res2['hits']

{'total': {'value': 70, 'relation': 'eq'},
 'max_score': 1.2085725,
 'hits': [{'_index': 'research_paper',
   '_id': '416f1ec4-bcbd-4204-9069-b6ea86c1b7a9_32_0',
   '_score': 1.2085725,
   '_source': {'document_id_universal': '416f1ec4-bcbd-4204-9069-b6ea86c1b7a9',
    'upload_date': 1704470257008,
    'language': 'en',
    'metadata': {'Author': '',
     'ModDate': '23-11-2023',
     'Keywords': '',
     'Title': '',
     'Subject': ''},
    'document_id_elastic': '416f1ec4-bcbd-4204-9069-b6ea86c1b7a9_32_0',
    'original_page_number': 32,
    'text_piece': 'Model\nOrca7B\n cautious sm\nOrca13B\n cautious sm\nOrca13B\nLLaMAChat13B\nWizardLM13B\nLLaMAChat70B\nWizardLM70B\nChatGPT\nGPT\nACIBENCH\nMS MARCO\nBLEU  PPL BLEU  PPL BLEU  PPL\n44\n2332\n11\n2652\n2903\n2542\n2313\n1945\nQMSum\n29\n80\n29\n37\n99\n21\n95\n17\n54\n49\n65\n25\n36\n67\n45\n41\n1027\n55\n1200\n2987\n2540\n1529\n2147\n2396\n2356\n2866\n2126\n2946\n36\n74\n41\n00\n35\n86\n84\n63\n00\n28\n83\n1190\n71\n58\n80\n46\n37\

In [26]:
import pdftitle
print(pdftitle.__file__)


/root/codebox39/lib/python3.10/site-packages/pdftitle.py


In [11]:
from pdftitle import get_title_from_file

In [3]:
pdf_path = '/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentBank/research_paper/416f1ec4-bcbd-4204-9069-b6ea86c1b7a9.pdf'

In [14]:
try:
    title = get_title_from_file(fp)
    print(f"The extracted title is: {title}")
except Exception as e:
    print(f"An error occurred: {e}")

The extracted title is: TODIRASCU Amalia


In [6]:
file_path = 'DocumentBank/research_paper/2610a7e7-359f-4e79-85d8-d3e3dff51dab.pdf'

In [2]:
fp= "/root/downloads/MY_CV_Final.pdf"

In [5]:
from DocumentManagement.documents import Document

SyntaxError: trailing comma not allowed without surrounding parentheses (pdf_processor.py, line 5)

In [3]:
doc = Document(file_path = 'DocumentBank/research_paper/2610a7e7-359f-4e79-85d8-d3e3dff51dab.pdf',document_type='research_paper',document_id='123')

NameError: name 'Document' is not defined

In [3]:
from DocumentProcessing.pdf_processing import pdf_processor

In [6]:
pdf_processor.extract_pdf_metadata(file_path)

{'Author': '',
 'CreationDate': 'D:20190528000751Z',
 'Creator': 'LaTeX with hyperref package',
 'Keywords': '',
 'ModDate': 'D:20190528000751Z',
 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2',
 'Producer': 'pdfTeX-1.40.17',
 'Subject': '',
 'Title': '',
 'Trapped': /'False'}

In [8]:
pdf_processor.structured_metadata_for_paper(file_path)

{'Author': '',
 'ModDate': '28-05-2019',
 'Keywords': '',
 'Subject': '',
 'Title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'}

In [1]:
from llama_index import SimpleDirectoryReader


In [7]:
from pathlib import Path
from llama_hub.file.unstructured import UnstructuredReader

loader = UnstructuredReader()
documents = loader.load_data(file=file_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
from llama_index import SimpleDirectoryReader
from llama_index.ingestion import IngestionPipeline
from llama_index.node_parser import TokenTextSplitter

In [33]:
pipeline = IngestionPipeline(transformations=[TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
])

In [38]:
from llama_index.text_splitter import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

In [34]:
nodes = pipeline.run(documents=documents)

476


In [39]:
from llama_index.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)


476


In [43]:
documents

[Document(id_='5d018435-7511-4201-a8ce-90611f265d7d', embedding=None, metadata={'filename': 'DocumentBank/research_paper/2610a7e7-359f-4e79-85d8-d3e3dff51dab.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='fddc03731566fb1a1bec9b091aaddec8ad0de836a005dc8e7253c9a7cf9a25fa', text='BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding\n\n{ jacobdevlin,mingweichang,kentonl,kristout } @google.com\n\nAbstract\n\nThere are two existing strategies for apply- ing pre-trained language representations to down- stream tasks: feature-based and ﬁne-tuning . The feature-based approach, such as ELMo (Peters et al., 2018a), uses task-speciﬁc architectures that include the pre-trained representations as addi- tional features. The ﬁne-tuning approach, such as the Generative Pre-trained Transformer (OpenAI GPT) (Radford et al., 2018), introduces minimal task-speciﬁc parameters, and is trained on the downstream tasks by simply ﬁne-tun

In [40]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex(nodes)


ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******